In [159]:
from typing import List
from functools import cache

class File:
    name: str
    size: int

    def __init__(self, name: str, size: int):
        self.name = name
        self.size = size

class Directory:
    name: str
    files: List[File]
    subdirs: List["Directory"]
    parent: "Directory"

    def __init__(self, name: str, parent: "Directory"):
        self.name = name
        self.files = []
        self.subdirs = []
        self.parent = parent

with open("input.txt", "r") as f:
    data = f.read().splitlines()

def parse(file) -> Directory:
    state = "commands"
    index = 1
    root = Directory("/", None)
    currDir = root

    while index < len(file):
        line = file[index]
        if state == "commands":
            match line.strip().split(" "):
                case ["$", "cd", dir]:
                    index -= 1
                    state = "cd"

                case ["$", "ls"]:
                    state = "ls"
        
        elif state == "cd":
            match line.strip().split(" "):
                case ["$", "cd", ".."]:
                    currDir = currDir.parent

                case ["$", "cd", dir]:
                    for directory in currDir.subdirs:
                        if directory.name == dir:
                            currDir = directory
                            break
            state = "commands"
                    
        elif state == "ls":
            match line.strip().split(" "):
                case ["dir", name]:
                    if name not in [directory.name for directory in currDir.subdirs]:
                        currDir.subdirs.append(Directory(name, currDir))

                case [value, name]:
                    if name not in [file.name for file in currDir.files]:
                        currDir.files.append(File(name, int(value)))
                
                case _:
                    state = "commands"
                    index -= 1
        index += 1

    return root

root = parse(data)

# function that returns the total size of all directories and files in a directory
@cache
def size(dir: Directory) -> int:
    total = sum(file.size for file in dir.files)
    for subdir in dir.subdirs:
        total += size(subdir)

    return total

# function that traverses the directory tree and returns a list of the sizes of each directory
def traverse(dir: Directory) -> List[int]:
    sizes = [size(dir)]
    for subdir in dir.subdirs:
        sizes += traverse(subdir)

    return sizes

result = [size for size in traverse(root) if size <= 100000]
sum(result)

1915606

In [162]:
list = traverse(root)
sorted_list = sorted(list, reverse=True)

current_space = 70000000 - sorted_list[0]
[i for i in sorted_list if current_space + i >= 30000000][-1]

5025657